# MODELO DEEP LEARNING

>1: Importación de bibliotecas

In [1]:
# Importar bibliotecas necesarias
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

>2: Cargar y barajar el conjunto de datos

In [2]:
# Cargar el conjunto de datos
df = pd.read_csv('dataset_procesado.csv')

# Barajar el conjunto de datos
df = shuffle(df, random_state=42)

>3: Preprocesamiento de datos

In [3]:
# Dividir el conjunto de datos en características (X) y etiquetas (y)
X = df.drop('income', axis=1)
y = df['income']

# Codificar las etiquetas ('<=50K' y '>50K') a valores numéricos
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

>4: Crear el modelo de red neuronal

In [4]:
# Crear el modelo de red neuronal
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

>5: Compilar el modelo

In [5]:
# Compilar el modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

>6: Entrenar el modelo

In [6]:
# Entrenar el modelo
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
651/651 [==============================] - 6s 6ms/step - loss: 0.3496 - accuracy: 0.8398 - val_loss: 0.3329 - val_accuracy: 0.8434
Epoch 2/10
651/651 [==============================] - 4s 6ms/step - loss: 0.3105 - accuracy: 0.8540 - val_loss: 0.3317 - val_accuracy: 0.8490
Epoch 3/10
651/651 [==============================] - 4s 5ms/step - loss: 0.3019 - accuracy: 0.8603 - val_loss: 0.3325 - val_accuracy: 0.8509
Epoch 4/10
651/651 [==============================] - 4s 6ms/step - loss: 0.2967 - accuracy: 0.8623 - val_loss: 0.3361 - val_accuracy: 0.8486
Epoch 5/10
651/651 [==============================] - 4s 6ms/step - loss: 0.2932 - accuracy: 0.8643 - val_loss: 0.3321 - val_accuracy: 0.8496
Epoch 6/10
651/651 [==============================] - 4s 6ms/step - loss: 0.2888 - accuracy: 0.8666 - val_loss: 0.3320 - val_accuracy: 0.8523
Epoch 7/10
651/651 [==============================] - 4s 6ms/step - loss: 0.2859 - accuracy: 0.8668 - val_loss: 0.3342 - val_accuracy: 0.8507
Epoch 

>7: Evaluar el modelo en el conjunto de prueba

In [7]:
# Evaluar el modelo en el conjunto de prueba
y_pred_probs = model.predict(X_test_scaled)
y_pred = (y_pred_probs > 0.5).astype(int)  # Redondear las probabilidades a 0 o 1

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy on test set: {accuracy}')

204/204 [==============================] - 1s 4ms/step
Accuracy on test set: 0.8523355869698832


>8: Guardar y cargar el modelo

In [11]:
# Guardar el modelo
model.save('my_model.h5')

# AJUSTE DE HIPERPARAMETROS con OPTUNA

In [28]:
import optuna

def objective(trial):
    # Parámetros a optimizar
    units_layer1 = trial.suggest_int('units_layer1', 32, 128)
    units_layer2 = trial.suggest_int('units_layer2', 16, 64)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-2)

    # Crear el modelo de red neuronal con parámetros sugeridos por Optuna
    model = Sequential()
    model.add(Dense(units_layer1, input_dim=X_train_scaled.shape[1], activation='relu'))
    model.add(Dense(units_layer2, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # Compilar el modelo con tasa de aprendizaje sugerida
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Entrenar el modelo
    model.fit(X_train_scaled, y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=0)

    # Evaluar el modelo
    y_pred_probs = model.predict(X_test_scaled)
    y_pred = (y_pred_probs > 0.5).astype(int)
    accuracy = accuracy_score(y_test, y_pred)

    return accuracy

# Crear un estudio de Optuna y realizar la optimización
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# Obtener los mejores hiperparámetros
best_params = study.best_params
best_accuracy = study.best_value

print(f'Best Hyperparameters: {best_params}')
print(f'Best Accuracy: {best_accuracy}')

c:\Users\Terán\Documents\finalProject\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2024-01-23 21:50:34,900] A new study created in memory with name: no-name-08a896ef-8fb6-47a3-a6ba-8f45921cee8d


204/204 [==============================] - 1s 2ms/step


[I 2024-01-23 21:50:46,932] Trial 0 finished with value: 0.8589428395820529 and parameters: {'units_layer1': 69, 'units_layer2': 22, 'learning_rate': 0.008235440331758503}. Best is trial 0 with value: 0.8589428395820529.


204/204 [==============================] - 1s 3ms/step


[I 2024-01-23 21:51:04,259] Trial 1 finished with value: 0.858020897357099 and parameters: {'units_layer1': 41, 'units_layer2': 21, 'learning_rate': 0.004767352622445229}. Best is trial 0 with value: 0.8589428395820529.


...

204/204 [==============================] - 1s 3ms/step


[I 2024-01-23 22:02:52,447] Trial 48 finished with value: 0.8544867854947756 and parameters: {'units_layer1': 32, 'units_layer2': 60, 'learning_rate': 0.008459850981752118}. Best is trial 3 with value: 0.8592501536570375.


204/204 [==============================] - 1s 2ms/step


[I 2024-01-23 22:03:07,826] Trial 49 finished with value: 0.852181929932391 and parameters: {'units_layer1': 50, 'units_layer2': 64, 'learning_rate': 0.00996459857254542}. Best is trial 3 with value: 0.8592501536570375.


Best Hyperparameters: {'units_layer1': 73, 'units_layer2': 52, 'learning_rate': 0.0043909237174682625}
Best Accuracy: 0.8592501536570375
